### [How to stream runnables](https://python.langchain.com/docs/how_to/streaming/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. LLMs and Chat Models

In [3]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')

In [4]:
chunks = []
for chunk in model.stream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| color| of| the| sky| can| vary| depending| on| the| time| of| day|,| weather| conditions|,| and| atmospheric| phenomena|.| Generally|,| during| a| clear| day|,| the| sky| appears| blue| due| to| the| scattering| of| sunlight| by| the| Earth's| atmosphere|.| During| sunrise| and| sunset|,| the| sky| can| take| on| shades| of| orange|,| pink|,| and| red|.| On| over|cast| days|,| it| may| appear| gray|,| and| at| night|,| the| sky| is| typically| dark|,| often| dotted| with| stars|.||

In [5]:
chunks = []
async for chunk in model.astream("what color is the sky?"):
    chunks.append(chunk)
    print(chunk.content, end='|', flush=True)

|The| color| of| the| sky| can| vary| depending| on| several| factors|,| including| the| time| of| day|,| weather| conditions|,| and| atmospheric| particles|.| During| a| clear| day|,| the| sky| typically| appears| blue| due| to| the| scattering| of| sunlight| by| the| atmosphere|.| At| sunrise| and| sunset|,| the| sky| can| display| a| range| of| colors|,| including| reds|,| oranges|,| and| pink|s|,| due| to| the| angle| of| the| sun| and| the| scattering| of| light|.| On| cloudy| or| over|cast| days|,| the| sky| may| appear| gray|.| Additionally|,| pollution| and| other| atmospheric| conditions| can| also| affect| the| perceived| color| of| the| sky|.||

In [6]:
chunks[0]

AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-2cbc5a7a-e00f-4a87-ae97-812197a23d7a')

In [7]:
chunks[0] + chunks[1] + chunks[2] + chunks[3] + chunks[4]

AIMessageChunk(content='The color of the', additional_kwargs={}, response_metadata={}, id='run-2cbc5a7a-e00f-4a87-ae97-812197a23d7a')

#### B. Chains

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
parser = StrOutputParser()
chain = prompt | model | parser


async for chunk in chain.astream({"topic": "parrot"}):
    print(chunk, end="|", flush=True)

|Why| did| the| par|rot| wear| a| rain|coat|?

|Because| it| wanted| to| be| a| poly|uns|aturated|!| 🦜|🌧|️||

### C. Working with input Streams

In [9]:
from langchain_core.output_parsers import JsonOutputParser

chain = (
    model | JsonOutputParser()
)

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, flush=True)

{}
{'countries': []}
{'countries': [{}]}
{'countries': [{'name': ''}]}
{'countries': [{'name': 'France'}]}
{'countries': [{'name': 'France', 'population': 652}]}
{'countries': [{'name': 'France', 'population': 652735}]}
{'countries': [{'name': 'France', 'population': 65273511}]}
{'countries': [{'name': 'France', 'population': 65273511}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': ''}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain'}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467547}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 46754778}, {}]}
{'countries': [{'name': 'France', 'population': 65273511}, {'name': 'Spain', 'population': 467

In [10]:
from langchain_core.output_parsers import JsonOutputParser

# A function that operates on finalized inputs
# rather than on an input_stream
def _extract_country_names(inputs):
    """A function that does not operates on input streams and breaks streaming."""
    if not isinstance(inputs, dict):
        return ""

    if "countries" not in inputs:
        return ""

    countries = inputs["countries"]

    if not isinstance(countries, list):
        return ""

    country_names = [
        country.get("name") for country in countries if isinstance(country, dict)
    ]
    return country_names

chain  = model | JsonOutputParser() | _extract_country_names

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`"
):
    print(text, end="|", flush=True)

['France', 'Spain', 'Japan']|

In [11]:
from langchain_core.output_parsers import JsonOutputParser

#! Funcion generadora, una funcion que usa yield
async def _extract_country_names_streaming(input_stream):
    """A function that operates on input streams."""
    country_names_so_far = set()

    async for input in input_stream:
        if not isinstance(input, dict):
            continue

        if "countries" not in input:
            continue

        countries = input["countries"]

        if not isinstance(countries, list):
            continue

        for country in countries:
            name = country.get("name")
            if not name:
                continue
            if name not in country_names_so_far:
                yield name
                country_names_so_far.add(name)

chain = model | JsonOutputParser() | _extract_country_names_streaming

async for text in chain.astream(
    "output a list of the countries france, spain and japan and their populations in JSON format. "
    'Use a dict with an outer key of "countries" which contains a list of countries. '
    "Each country should have the key `name` and `population`",
):
    print(text, end="|", flush=True)

France|Spain|Japan|

#### D. Non-streaming components

In [14]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho", "harrison likes spicy food"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

chunks = [chunk for chunk in retriever.stream("where did harrison work?")]
chunks

[[Document(id='6a306925-fd52-406c-ae07-7f8f0ee2646c', metadata={}, page_content='harrison worked at kensho'),
  Document(id='64933687-3d7f-43f1-998c-b4869aee8853', metadata={}, page_content='harrison likes spicy food')]]

In [15]:
retrieval_chain = (
    {
        "context": retriever.with_config(run_name="Docs"),
        "question": RunnablePassthrough(),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [16]:
for chunk in retrieval_chain.stream(
    "Where did harrison work? " "Write 3 made up sentences about this place."
):
    print(chunk, end="|", flush=True)

|H|arrison| worked| at| Kens|ho|.| Kens|ho| is| renowned| for| its| innovative| approach| to| data| analysis|,| leveraging| cutting|-edge| technology| to| provide| insights| that| drive| business| decisions|.| The| vibrant| office| culture| at| Kens|ho| encourages| creativity| and| collaboration|,| making| it| a| sought|-after| workplace| for| tech| enthusiasts|.| Additionally|,| the| company| hosts| regular| team|-building| events| that| foster| strong| relationships| among| employees|,| enhancing| overall| productivity| and| job| satisfaction|.||

#### E. Using Stream events & Event Reference 
https://python.langchain.com/docs/how_to/streaming/#using-stream-events

In [17]:
import langchain_core

langchain_core.__version__

'0.3.28'